In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
pip install tensorflow==2.1.0

In [0]:
pip install tensorflow-gpu

In [0]:
!ls "/content/drive/My Drive/Machine_Learning/sign_images" -a

00000  00001  00002  00003  00004


In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
from PIL import Image
import numpy as np
import os

train_images = []
train_labels = []
count = -1
for dirc in sorted(os.listdir('/content/drive/My Drive/Machine_Learning/sign_images')):
    if (count < 0):
        count += 1
        continue
    if (count > 2):
        continue
    for img_path in os.listdir('/content/drive/My Drive/Machine_Learning/sign_images/' + str(dirc)):
        try:
            im = Image.open('/content/drive/My Drive/Machine_Learning/sign_images/' + str(dirc) + "/" + str(img_path))
            im = im.resize((100, 100))
            train_images.append(np.array(im))
            train_labels.append(count - 1)
        except:
            continue
    count += 1
    print('Reading ' + str(dirc))

In [0]:
np.unique(train_labels)

In [0]:
np.shape(train_labels)

(4480,)

In [0]:
train_labels = np.reshape(train_labels, (-1, 1))
np.shape(train_labels)

(4480, 1)

In [0]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
train_labels = enc.fit_transform(train_labels).toarray()

In [0]:
np.shape(train_labels)

(4480, 2)

In [0]:
np.shape(train_images)

(4480, 100, 100, 3)

In [0]:
train_images = np.array(train_images) / 255.0

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)
train_ds

<BatchDataset shapes: ((None, 100, 100, 3), (None, 2)), types: (tf.float64, tf.float64)>

In [0]:
del train_images
del train_labels
import gc
gc.collect()

In [0]:
class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same', activation='relu')
        self.pool1 = MaxPool2D((2, 2))
        self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
        self.pool2 = MaxPool2D((2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(512, activation='relu')
        self.dropout1 = Dropout(0.4)
        self.d2 = Dense(128, activation='relu')
        self.dropout2 = Dropout(0.4)
        self.d3 = Dense(2, activation='softmax')
    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout1(x)
        x = self.d2(x)
        x = self.dropout2(x)
        x = self.d3(x)
        return x

model = CNNModel()

In [0]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [0]:
epochs = 10
for epoch in range(epochs):
    for images, labels in train_ds:
        train_step(images, labels)
    model.save_weights('saved_models', save_format='tf')
    print('Epoch: ' + str(epoch + 1) + ' Loss: ' + str(train_loss.result()), ' Acc: ' + str(train_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch: 1 Loss: tf.Tensor(0.022242397, shape=(), dtype=float32)  Acc: tf.Tensor(99.21875, shape=(), dtype=float32)
Epoch: 2 Loss: tf.Tensor(0.02025445, shape=(), dtype=float32)  Acc: tf.Tensor(99.41964, shape=(), dtype=float32)
Epoch: 3 Loss: tf.Tensor(0.02292676, shape=(), dtype=float32)  Acc: tf.Tensor(99.040184, shape=(), dtype=float32)
Epoch: 4 Loss: tf.Tensor(0.024775414, shape=(), dtype=float32)  Acc: tf.Tensor(99.1741, shape=(), dtype=float32)
Epoch: 5 Loss: tf.Tensor(0.02421321, shape=(), dtype=float32)  Acc: tf.Tensor(99.24107, shape=(), dtype=float32)
Epoch: 6 Loss: tf.Tensor(0.016320234, shape=(), dtype=float32)  Acc: tf.Tensor(99.48661, shape=(), dtype=float32)
Epoch: 7 Loss: tf.Tensor(0.0013564806, shape=(), dtype=float32)  Acc: tf.Tensor(100.0, shape=(), dtype=float32)
Epoch: 8 Loss: tf.Tensor(0.00019864377, shape=(), dtype=float32)  Acc: tf.Tensor(100.0, shape=(), dtype=float32)
Epoch: 9 Loss: tf.Tensor(8.018793e-05, shape=(), dtype=float32)  Acc: tf.Tensor(100.0, shape=(